What is the `compose_combinator`?

!! It is only meant to be use by library authors. It is used to implement other combinators such as `switch_combinator`, the joint `mixture_combinator`, and `repeat_combinator`.

In [13]:
# Example of rewriting the `mixture_combinator` using the `compose_combinator` and `switch_combinator` functions
from genjax._src.core.generative import GenerativeFunction
from genjax._src.generative_functions.combinators.compose_combinator import (
    compose_combinator,
)
from genjax._src.generative_functions.combinators.switch_combinator import (
    switch_combinator,
)
from genjax import categorical, gen, normal, mixture_combinator
import jax.random as random

def new_mixture_combinator(
    *gen_fns: GenerativeFunction,
) -> GenerativeFunction:
    def argument_mapping(mixture_logits, *args):  
        return (mixture_logits, *args)

    # Packing the generative functions using a switch combinator
    inner_combinator_closure = switch_combinator(*gen_fns) 

    @gen
    def mixture_model(mixture_logits, *args): 
        # The mixture combinator creates a mixture model that samples from a categorical distribution to get the index of the component to sample from
        mix_idx = categorical(logits=mixture_logits) @ "mixture_component"
        # And then calls the inner combinator closure to sample from the selected component
        v = inner_combinator_closure(mix_idx, *args) @ "component_sample"
        return v

    # The compose_combinator function is used to wrap the mixture model function with the argument mapping function to create a new generative function
    return compose_combinator(
        mixture_model,
        pre=argument_mapping,
        info="Derived combinator (Mixture)",
    )

# Testing the rewritten version on an example
@gen
def mixture_model(p):
    logits = (0.4, 0.6)
    arg_1 = (p,)
    arg_2 = (p,)
    component_1 = gen(lambda p: normal(p, 0.1) @ "x")
    component_2 = gen(lambda p: normal(p, 0.2) @ "y")
    a = new_mixture_combinator(component_1,component_2)(logits, arg_1, arg_2) @ "a"
    return a
    
key = random.PRNGKey(23)
tr = mixture_model.simulate(key, (0.4,))
print(tr.get_retval())
    

0.14682357
